Copyright (C) 2025 Advanced Micro Devices, Inc. All rights reserved.
Portions of this notebook consist of AI-generated content.

Permission is hereby granted, free of charge, to any person obtaining a copy

of this software and associated documentation files (the "Software"), to deal

in the Software without restriction, including without limitation the rights

to use, copy, modify, merge, publish, distribute, sublicense, and/or sell

copies of the Software, and to permit persons to whom the Software is

furnished to do so, subject to the following conditions:



The above copyright notice and this permission notice shall be included in all

copies or substantial portions of the Software.



THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR

IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,

FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE

AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER

LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,

OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE

SOFTWARE.

# Lab 3: Neural Network Fundamentals - Linear Layers and Tensor Operations

## Lab Overview

This lab explores the fundamental building blocks of neural networks, focusing on linear transformations, tensor operations, and the mathematical foundations that power transformer architectures.

## Learning Objectives

By the end of this lab, you will:
- Understand linear layers and their role in neural networks
- Master tensor operations and broadcasting in PyTorch
- Learn about weight matrices and bias vectors
- Explore matrix multiplication in the context of neural networks
- Understand how linear transformations work in transformer models

---

## 1. Environment Setup


In [2]:
# Initialize AMD GPU Backend and Required Libraries
import sys

sys.path.append("../")

# Core libraries for neural network fundamentals
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

Using device: cuda
PyTorch version: 2.9.1+rocm7.10.0
GPU: Radeon 8060S Graphics
GPU Memory: 103.1 GB


## 2. Understanding Linear Layers

Linear layers (also called fully connected or dense layers) are fundamental components of neural networks. They perform the operation: **y = xW^T + b**

**Components:**
- **Weight Matrix (W)**: Learnable parameters that transform inputs
- **Bias Vector (b)**: Learnable offset added to the transformation
- **Input (x)**: Data to be transformed
- **Output (y)**: Transformed result

**In Transformers:**
- Query, Key, Value projections in attention
- Feed-forward network layers
- Output projection layers


In [3]:
# Create a linear layer: 2 input features -> 3 output features
linear = nn.Linear(in_features=2, out_features=3)

# Move to AMD GPU
linear = linear.to(device)

print("Linear Layer Analysis:")
print(f"Input features: {linear.in_features}")
print(f"Output features: {linear.out_features}")
print(f"Weight matrix shape: {linear.weight.shape}")  # [out_features, in_features]
print(f"Bias vector shape: {linear.bias.shape}")  # [out_features]
print(f"Device: {linear.weight.device}")

print("\nWeight Matrix:")
print(linear.weight.data)
print("\nBias Vector:")
print(linear.bias.data)

# Count parameters
total_params = sum(p.numel() for p in linear.parameters())
print(f"\nTotal parameters: {total_params}")
print(f"   Weights: {linear.weight.numel()}")
print(f"   Bias: {linear.bias.numel()}")

Linear Layer Analysis:
Input features: 2
Output features: 3
Weight matrix shape: torch.Size([3, 2])
Bias vector shape: torch.Size([3])
Device: cuda:0

Weight Matrix:
tensor([[ 0.5406,  0.5869],
        [-0.1657,  0.6496],
        [-0.1549,  0.1427]], device='cuda:0')

Bias Vector:
tensor([-0.3443,  0.4153,  0.6233], device='cuda:0')

Total parameters: 9
   Weights: 6
   Bias: 3


## 3: Tensor Operations and Matrix Multiplication

Now let's explore how linear layers process multi-dimensional tensors and understand the mathematical operations involved.

**Key Operations:**
1. **Matrix Multiplication**: Core operation in linear layers
2. **Tensor Broadcasting**: Automatic dimension expansion for operations
3. **Batch Processing**: Handling multiple samples simultaneously
4. **GPU Acceleration**: Parallel computation on AMD hardware

**Mathematical Details:**
- For input `x` with shape `[batch_size, ..., in_features]`
- Weight `W` with shape `[out_features, in_features]`
- Output `y = x @ W.T + b` with shape `[batch_size, ..., out_features]`

In [4]:
# Create a multi-dimensional input tensor (simulating batch processing)
# Shape: [batch_size=2, seq_length=2, height=2, in_features=2]
x = torch.randn(2, 2, 2, 2, device=device)
print("Input Tensor Analysis:")
print(f"Input shape: {x.shape}")
print(f"Input tensor:\n{x}")

# Apply linear transformation
y = linear(x)
print("\nOutput Analysis:")
print(f"Output shape: {y.shape}")  # [2, 2, 2, 3]
print(f"Output tensor:\n{y}")

print("\nManual Computation Verification:")
# Manual matrix multiplication to understand the operation
# Last dimension is transformed: [..., 2] -> [..., 3]

# Method 1: Using torch.matmul
y1 = torch.matmul(x, linear.weight.t())  # x @ W.T
print(f"After matrix multiplication shape: {y1.shape}")

# Add bias (broadcasting automatically handles the shape)
y2 = y1 + linear.bias
print(f"After adding bias shape: {y2.shape}")

# Verify our manual computation matches PyTorch's linear layer
print("\nVerification:")
print(f"Manual computation matches linear layer: {torch.allclose(y, y2, atol=1e-6)}")

# Show detailed computation for one element
print("\nDetailed Example (first element):")
print(f"Input [0,0,0,:]: {x[0, 0, 0, :]}")
print(f"Weight matrix:\n{linear.weight}")
print(f"Bias: {linear.bias}")
single_result = x[0, 0, 0, :] @ linear.weight.t() + linear.bias
print(f"Manual result: {single_result}")
print(f"Linear layer result: {y[0, 0, 0, :]}")
print(f"Match: {torch.allclose(single_result, y[0, 0, 0, :], atol=1e-6)}")

Input Tensor Analysis:
Input shape: torch.Size([2, 2, 2, 2])
Input tensor:
tensor([[[[ 0.1940,  2.1614],
          [-0.1721,  0.8491]],

         [[-1.9244,  0.6530],
          [-0.6494, -0.8175]]],


        [[[ 0.5280, -1.2753],
          [-1.6621, -0.3033]],

         [[-0.0926,  0.1992],
          [-1.1204,  1.8577]]]], device='cuda:0')

Output Analysis:
Output shape: torch.Size([2, 2, 2, 3])
Output tensor:
tensor([[[[ 1.0291,  1.7871,  0.9017],
          [ 0.0610,  0.9953,  0.7712]],

         [[-1.0014,  1.1582,  1.0147],
          [-1.1752, -0.0082,  0.6073]]],


        [[[-0.8073, -0.5006,  0.3596],
          [-1.4208,  0.4936,  0.8376]],

         [[-0.2774,  0.5600,  0.6661],
          [ 0.1403,  1.8075,  1.0620]]]], device='cuda:0',
       grad_fn=<ViewBackward0>)

Manual Computation Verification:
After matrix multiplication shape: torch.Size([2, 2, 2, 3])
After adding bias shape: torch.Size([2, 2, 2, 3])

Verification:
Manual computation matches linear layer: True

Detaile

## 4: Tensor Broadcasting

Broadcasting is a powerful feature that allows operations between tensors of different shapes. It's crucial for efficient neural network computations.

**Broadcasting Rules:**
1. Align shapes from the rightmost dimension
2. Dimensions of size 1 are stretched to match
3. Missing dimensions are treated as size 1
4. Incompatible shapes raise errors

**Applications in Neural Networks:**
- Adding bias vectors to matrix multiplication results
- Element-wise operations with different shaped tensors
- Scaling and normalization operations

In [5]:
# Tensor Broadcasting Examples
print("Broadcasting Examples:")

# Example 1: Scalar broadcasting
a = torch.randn(2, 3, 4, device=device)
b = 10.0  # scalar

print(f"Tensor a shape: {a.shape}")
print(f"Scalar b: {b}")

# Broadcasting: scalar is automatically expanded
result1 = a + b
print(f"a + b result shape: {result1.shape}")
print(f"First element: a[0,0,0] = {a[0, 0, 0]:.3f}, result[0,0,0] = {result1[0, 0, 0]:.3f}")

print("\n" + "=" * 60)

# Example 2: Vector broadcasting
c = torch.randn(2, 3, 1, device=device)  # Can broadcast to (2, 3, 4)
d = torch.randn(4, device=device)  # Can broadcast to (2, 3, 4)

print(f"Tensor c shape: {c.shape}")
print(f"Tensor d shape: {d.shape}")

result2 = c + d  # Broadcasting happens automatically
print(f"c + d result shape: {result2.shape}")

print("\nBroadcasting visualization:")
print(f"c[0,0,0] = {c[0, 0, 0].item():.3f} gets added to each element of d")
print(f"d = {d}")
print(f"Result c[0,0,:] + d = {result2[0, 0, :]}")

print("\n" + "=" * 60)

# Example 3: Matrix broadcasting (common in attention mechanisms)
# Simulate attention scores + positional bias
attention_scores = torch.randn(2, 4, 8, 8, device=device)  # [batch, heads, seq, seq]
positional_bias = torch.randn(1, 1, 8, 8, device=device)  # [1, 1, seq, seq]

print(f"Attention scores shape: {attention_scores.shape}")
print(f"Positional bias shape: {positional_bias.shape}")

# Broadcasting automatically handles different batch/head dimensions
attention_with_bias = attention_scores + positional_bias
print(f"Result shape: {attention_with_bias.shape}")

print("\nBroadcasting enables efficient operations without explicit tensor reshaping!")

# Example 4: What happens without broadcasting (manual expansion)
print("\nManual expansion vs Broadcasting:")
positional_bias_expanded = positional_bias.expand_as(attention_scores)
print(f"Manually expanded bias shape: {positional_bias_expanded.shape}")
print(f"Memory usage - Original bias: {positional_bias.numel()} elements")
print(f"Memory usage - Expanded bias: {positional_bias_expanded.numel()} elements")
print("Broadcasting saves memory by not creating copies!")

Broadcasting Examples:
Tensor a shape: torch.Size([2, 3, 4])
Scalar b: 10.0
a + b result shape: torch.Size([2, 3, 4])
First element: a[0,0,0] = 0.139, result[0,0,0] = 10.139

Tensor c shape: torch.Size([2, 3, 1])
Tensor d shape: torch.Size([4])
c + d result shape: torch.Size([2, 3, 4])

Broadcasting visualization:
c[0,0,0] = -0.519 gets added to each element of d
d = tensor([-0.6974, -1.8688, -0.8832, -1.6627], device='cuda:0')
Result c[0,0,:] + d = tensor([-1.2161, -2.3876, -1.4019, -2.1815], device='cuda:0')

Attention scores shape: torch.Size([2, 4, 8, 8])
Positional bias shape: torch.Size([1, 1, 8, 8])
Result shape: torch.Size([2, 4, 8, 8])

Broadcasting enables efficient operations without explicit tensor reshaping!

Manual expansion vs Broadcasting:
Manually expanded bias shape: torch.Size([2, 4, 8, 8])
Memory usage - Original bias: 64 elements
Memory usage - Expanded bias: 512 elements
Broadcasting saves memory by not creating copies!


## 5: Practical Applications in Transformers

Let's see how these linear operations are used in transformer architectures with practical examples.

**Transformer Applications:**
1. **Attention Projections**: Q, K, V matrices
2. **Feed-Forward Networks**: Two linear layers with activation
3. **Output Projection**: Final linear layer for vocabulary prediction
4. **Layer Normalization**: Learned scale and shift parameters

In [6]:
# Simulate transformer components using linear layers

print("Transformer Component Simulation:")

# Parameters for a mini transformer
embed_dim = 64
seq_length = 8
batch_size = 2

# Simulate input embeddings
x = torch.randn(batch_size, seq_length, embed_dim, device=device)
print(f"Input embeddings shape: {x.shape}")

print("\n" + "=" * 50)
print("1. ATTENTION PROJECTIONS:")

# Query, Key, Value projections (as used in self-attention)
q_proj = nn.Linear(embed_dim, embed_dim, bias=False).to(device)
k_proj = nn.Linear(embed_dim, embed_dim, bias=False).to(device)
v_proj = nn.Linear(embed_dim, embed_dim, bias=False).to(device)

# Apply projections
Q = q_proj(x)  # Query
K = k_proj(x)  # Key
V = v_proj(x)  # Value

print(f"Query (Q) shape: {Q.shape}")
print(f"Key (K) shape: {K.shape}")
print(f"Value (V) shape: {V.shape}")

# Attention scores (simplified - no multi-head)
attention_scores = torch.matmul(Q, K.transpose(-2, -1))
print(f"Attention scores shape: {attention_scores.shape}")

print("\n" + "=" * 50)
print("2. FEED-FORWARD NETWORK:")

# Two-layer MLP as used in transformers
mlp_dim = embed_dim * 4  # Common practice: 4x expansion
ff_layer1 = nn.Linear(embed_dim, mlp_dim).to(device)
ff_layer2 = nn.Linear(mlp_dim, embed_dim).to(device)

# Forward pass through MLP
ff_out1 = ff_layer1(x)
ff_out1_activated = F.relu(ff_out1)  # ReLU activation
ff_out2 = ff_layer2(ff_out1_activated)

print(f"Input to MLP: {x.shape}")
print(f"After first linear layer: {ff_out1.shape}")
print(f"After activation: {ff_out1_activated.shape}")
print(f"After second linear layer: {ff_out2.shape}")

print("\n" + "=" * 50)
print("3. OUTPUT PROJECTION:")

# Final projection to vocabulary (e.g., for language modeling)
vocab_size = 10000
output_proj = nn.Linear(embed_dim, vocab_size).to(device)

# Apply output projection
logits = output_proj(x)
print(f"Final logits shape: {logits.shape}")  # [batch, seq_length, vocab_size]

print("\nAll operations completed efficiently on AMD GPU!")

# Parameter counting
total_params = 0
for name, layer in [
    ("Q proj", q_proj),
    ("K proj", k_proj),
    ("V proj", v_proj),
    ("FF layer 1", ff_layer1),
    ("FF layer 2", ff_layer2),
    ("Output proj", output_proj),
]:
    params = sum(p.numel() for p in layer.parameters())
    total_params += params
    print(f"{name}: {params:,} parameters")

print(f"\nTotal parameters: {total_params:,}")
print(f"Memory usage: ~{total_params * 4 / 1024**2:.1f} MB (float32)")

Transformer Component Simulation:
Input embeddings shape: torch.Size([2, 8, 64])

1. ATTENTION PROJECTIONS:
Query (Q) shape: torch.Size([2, 8, 64])
Key (K) shape: torch.Size([2, 8, 64])
Value (V) shape: torch.Size([2, 8, 64])
Attention scores shape: torch.Size([2, 8, 8])

2. FEED-FORWARD NETWORK:
Input to MLP: torch.Size([2, 8, 64])
After first linear layer: torch.Size([2, 8, 256])
After activation: torch.Size([2, 8, 256])
After second linear layer: torch.Size([2, 8, 64])

3. OUTPUT PROJECTION:
Final logits shape: torch.Size([2, 8, 10000])

All operations completed efficiently on AMD GPU!
Q proj: 4,096 parameters
K proj: 4,096 parameters
V proj: 4,096 parameters
FF layer 1: 16,640 parameters
FF layer 2: 16,448 parameters
Output proj: 650,000 parameters

Total parameters: 695,376
Memory usage: ~2.7 MB (float32)


## Lab Summary

### Technical Concepts Learned
- **Linear Transformations**: The mathematical foundation y = xW^T + b
- **Parameter Management**: Weight initialization and device placement
- **Batch Processing**: Handling multiple samples simultaneously
- **Memory Efficiency**: Broadcasting vs explicit tensor expansion
- **Transformer Architecture**: How linear layers build complex models

### Experiment Further
Try these modifications to deepen your understanding:
- Change embedding dimensions and observe parameter scaling
- Experiment with different tensor shapes and broadcasting
- Compare CPU vs GPU computation times
- Implement custom linear layers from scratch